In [1]:
import pandas as pd
import numpy as np
import nltk
import pickle

In [2]:
train_path = '/Users/kelvin/Toxicity/data/train.csv'
test_path = '/Users/kelvin/Toxicity/data/test.csv'

In [3]:
df = pd.read_csv(train_path)

In [4]:
df.target[df.target >= 0.5]
text = df.comment_text
df['class_target'] = np.where(df['target'] >= 0.5, 'toxic', 'non_toxic')

In [5]:
df.class_target.value_counts(normalize=True)*100

non_toxic    92.003098
toxic         7.996902
Name: class_target, dtype: float64

In [2]:
with open('preprocess_data.pickle', mode='rb') as f:
    data = pickle.load(f)

In [4]:
# gensim
from gensim import corpora, models, similarities, matutils

# sklearn
from sklearn import datasets
from sklearn.feature_extraction.text import CountVectorizer

# logging for gensim (set to INFO)
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [5]:
# Create a CountVectorizer for parsing/counting words
count_vectorizer = CountVectorizer(stop_words='english', token_pattern="\\b[a-z][a-z]+\\b")
count_vectorizer.fit(data.comment_text)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words='english',
                strip_accents=None, token_pattern='\\b[a-z][a-z]+\\b',
                tokenizer=None, vocabulary=None)

In [6]:
# Create the term-document matrix
# Transpose it so the terms are the rows
doc_word = count_vectorizer.transform(data.comment_text).transpose()

In [7]:
# Convert sparse matrix of counts to a gensim corpus
corpus = matutils.Sparse2Corpus(doc_word)

In [8]:
id2word = dict((v, k) for k, v in count_vectorizer.vocabulary_.items())

In [9]:
lda = models.LdaModel(corpus=corpus, num_topics=6, id2word=id2word, passes=5)

2019-11-13 15:32:49,164 : INFO : using symmetric alpha at 0.16666666666666666
2019-11-13 15:32:49,169 : INFO : using symmetric eta at 0.16666666666666666
2019-11-13 15:32:49,229 : INFO : using serial LDA version on this node
2019-11-13 15:32:49,579 : INFO : running online (multi-pass) LDA training, 6 topics, 5 passes over the supplied corpus of 1804874 documents, updating model once every 2000 documents, evaluating perplexity every 20000 documents, iterating 50x with a convergence threshold of 0.001000
2019-11-13 15:32:49,612 : INFO : PROGRESS: pass 0, at document #2000/1804874
2019-11-13 15:32:51,792 : INFO : merging changes from 2000 documents into a model of 1804874 documents
2019-11-13 15:32:51,906 : INFO : topic #3 (0.167): 0.011*"people" + 0.007*"just" + 0.005*"think" + 0.005*"like" + 0.004*"comments" + 0.004*"oregon" + 0.004*"public" + 0.004*"government" + 0.003*"time" + 0.003*"know"
2019-11-13 15:32:51,909 : INFO : topic #0 (0.167): 0.008*"does" + 0.005*"people" + 0.005*"like" 

2019-11-13 15:32:59,832 : INFO : PROGRESS: pass 0, at document #14000/1804874
2019-11-13 15:33:01,121 : INFO : merging changes from 2000 documents into a model of 1804874 documents
2019-11-13 15:33:01,205 : INFO : topic #2 (0.167): 0.010*"people" + 0.007*"like" + 0.007*"city" + 0.006*"just" + 0.006*"time" + 0.006*"good" + 0.005*"think" + 0.005*"eugene" + 0.004*"work" + 0.004*"public"
2019-11-13 15:33:01,209 : INFO : topic #4 (0.167): 0.015*"alaska" + 0.009*"just" + 0.008*"people" + 0.007*"like" + 0.007*"law" + 0.005*"did" + 0.005*"time" + 0.004*"know" + 0.004*"got" + 0.004*"does"
2019-11-13 15:33:01,213 : INFO : topic #1 (0.167): 0.012*"like" + 0.008*"trump" + 0.007*"just" + 0.006*"does" + 0.006*"bernie" + 0.006*"vote" + 0.005*"party" + 0.005*"did" + 0.005*"sanders" + 0.005*"republican"
2019-11-13 15:33:01,219 : INFO : topic #5 (0.167): 0.016*"people" + 0.008*"just" + 0.007*"like" + 0.006*"did" + 0.006*"right" + 0.006*"know" + 0.005*"think" + 0.005*"good" + 0.004*"want" + 0.004*"time"


2019-11-13 15:33:09,517 : INFO : topic #5 (0.167): 0.015*"people" + 0.008*"just" + 0.007*"know" + 0.007*"did" + 0.006*"like" + 0.005*"think" + 0.005*"right" + 0.005*"good" + 0.005*"time" + 0.004*"public"
2019-11-13 15:33:09,521 : INFO : topic #3 (0.167): 0.010*"government" + 0.009*"people" + 0.006*"just" + 0.006*"state" + 0.005*"think" + 0.004*"federal" + 0.004*"like" + 0.004*"public" + 0.003*"political" + 0.003*"time"
2019-11-13 15:33:09,526 : INFO : topic #1 (0.167): 0.011*"like" + 0.008*"vote" + 0.008*"just" + 0.007*"republican" + 0.006*"trump" + 0.006*"state" + 0.006*"party" + 0.005*"legislators" + 0.005*"did" + 0.005*"majority"
2019-11-13 15:33:09,531 : INFO : topic #4 (0.167): 0.034*"alaska" + 0.009*"just" + 0.008*"law" + 0.007*"people" + 0.007*"like" + 0.006*"did" + 0.005*"time" + 0.005*"man" + 0.005*"does" + 0.004*"know"
2019-11-13 15:33:09,536 : INFO : topic diff=0.081781, rho=0.277350
2019-11-13 15:33:09,618 : INFO : PROGRESS: pass 0, at document #28000/1804874
2019-11-13 15:

2019-11-13 15:33:15,808 : INFO : topic #1 (0.167): 0.010*"like" + 0.009*"vote" + 0.007*"just" + 0.007*"republican" + 0.006*"legislators" + 0.006*"majority" + 0.006*"party" + 0.005*"state" + 0.005*"did" + 0.005*"does"
2019-11-13 15:33:15,812 : INFO : topic diff=0.057429, rho=0.229416
2019-11-13 15:33:17,188 : INFO : -9.063 per-word bound, 534.9 perplexity estimate based on a held-out corpus of 2000 documents with 50646 words
2019-11-13 15:33:17,189 : INFO : PROGRESS: pass 0, at document #40000/1804874
2019-11-13 15:33:18,052 : INFO : merging changes from 2000 documents into a model of 1804874 documents
2019-11-13 15:33:18,147 : INFO : topic #4 (0.167): 0.017*"alaska" + 0.010*"dog" + 0.010*"gun" + 0.008*"law" + 0.008*"just" + 0.008*"people" + 0.007*"like" + 0.007*"did" + 0.006*"know" + 0.006*"bear"
2019-11-13 15:33:18,152 : INFO : topic #1 (0.167): 0.011*"like" + 0.009*"vote" + 0.007*"just" + 0.006*"legislators" + 0.006*"republican" + 0.006*"majority" + 0.006*"party" + 0.005*"state" + 0.

2019-11-13 15:33:24,267 : INFO : merging changes from 2000 documents into a model of 1804874 documents
2019-11-13 15:33:24,368 : INFO : topic #2 (0.167): 0.009*"city" + 0.007*"just" + 0.007*"www" + 0.007*"http" + 0.007*"com" + 0.007*"time" + 0.006*"like" + 0.006*"people" + 0.006*"good" + 0.005*"eugene"
2019-11-13 15:33:24,372 : INFO : topic #0 (0.167): 0.029*"state" + 0.026*"oil" + 0.020*"tax" + 0.019*"alaska" + 0.016*"money" + 0.016*"pay" + 0.008*"taxes" + 0.008*"income" + 0.007*"people" + 0.007*"companies"
2019-11-13 15:33:24,376 : INFO : topic #1 (0.167): 0.013*"trump" + 0.011*"vote" + 0.011*"like" + 0.008*"party" + 0.008*"just" + 0.007*"republican" + 0.005*"people" + 0.005*"republicans" + 0.005*"legislators" + 0.005*"does"
2019-11-13 15:33:24,385 : INFO : topic #3 (0.167): 0.009*"people" + 0.007*"government" + 0.005*"just" + 0.004*"state" + 0.004*"public" + 0.004*"like" + 0.004*"rights" + 0.004*"land" + 0.004*"property" + 0.003*"federal"
2019-11-13 15:33:24,390 : INFO : topic #5 (0

...

2019-11-13 15:53:57,034 : INFO : PROGRESS: pass 1, at document #672000/1804874
2019-11-13 15:53:57,660 : INFO : merging changes from 2000 documents into a model of 1804874 documents
2019-11-13 15:53:57,742 : INFO : topic #1 (0.167): 0.043*"trump" + 0.011*"president" + 0.009*"did" + 0.008*"like" + 0.008*"news" + 0.008*"trudeau" + 0.007*"media" + 0.007*"just" + 0.007*"obama" + 0.007*"party"
2019-11-13 15:53:57,746 : INFO : topic #0 (0.167): 0.015*"money" + 0.014*"tax" + 0.011*"pay" + 0.009*"government" + 0.009*"people" + 0.008*"canada" + 0.007*"state" + 0.007*"taxes" + 0.007*"oil" + 0.006*"jobs"
2019-11-13 15:53:57,753 : INFO : topic #4 (0.167): 0.012*"people" + 0.011*"like" + 0.009*"just" + 0.008*"did" + 0.006*"police" + 0.005*"know" + 0.005*"man" + 0.005*"muslims" + 0.005*"right" + 0.005*"white"
2019-11-13 15:53:57,757 : INFO : topic #3 (0.167): 0.013*"canada" + 0.012*"law" + 0.008*"immigration" + 0.008*"country" + 0.007*"countries" + 0.007*"illegal" + 0.007*"government" + 0.007*"state

2019-11-13 15:54:03,733 : INFO : topic #2 (0.167): 0.008*"like" + 0.008*"just" + 0.007*"time" + 0.007*"years" + 0.006*"good" + 0.006*"city" + 0.006*"com" + 0.005*"www" + 0.005*"canadians" + 0.004*"new"
2019-11-13 15:54:03,737 : INFO : topic #3 (0.167): 0.013*"canada" + 0.012*"law" + 0.008*"immigration" + 0.007*"country" + 0.007*"illegal" + 0.007*"government" + 0.007*"state" + 0.007*"countries" + 0.006*"states" + 0.005*"rights"
2019-11-13 15:54:03,742 : INFO : topic #5 (0.167): 0.012*"people" + 0.008*"does" + 0.007*"think" + 0.006*"just" + 0.006*"know" + 0.005*"like" + 0.005*"read" + 0.005*"article" + 0.005*"make" + 0.004*"good"
2019-11-13 15:54:03,746 : INFO : topic #0 (0.167): 0.015*"money" + 0.015*"tax" + 0.011*"pay" + 0.009*"government" + 0.009*"people" + 0.008*"canada" + 0.007*"taxes" + 0.007*"state" + 0.006*"cost" + 0.006*"oil"
2019-11-13 15:54:03,750 : INFO : topic diff=0.006647, rho=0.033251
2019-11-13 15:54:03,780 : INFO : PROGRESS: pass 1, at document #686000/1804874
2019-11-1

2019-11-13 15:54:09,744 : INFO : topic #4 (0.167): 0.012*"people" + 0.010*"like" + 0.008*"just" + 0.008*"did" + 0.006*"man" + 0.006*"know" + 0.005*"police" + 0.005*"right" + 0.005*"muslims" + 0.004*"white"
2019-11-13 15:54:09,750 : INFO : topic #2 (0.167): 0.008*"like" + 0.008*"just" + 0.007*"time" + 0.007*"years" + 0.006*"com" + 0.006*"good" + 0.006*"city" + 0.006*"www" + 0.005*"canadians" + 0.004*"http"
2019-11-13 15:54:09,756 : INFO : topic diff=0.007654, rho=0.033251
2019-11-13 15:54:09,798 : INFO : PROGRESS: pass 1, at document #698000/1804874
2019-11-13 15:54:11,322 : INFO : merging changes from 2000 documents into a model of 1804874 documents
2019-11-13 15:54:11,445 : INFO : topic #5 (0.167): 0.011*"people" + 0.008*"does" + 0.007*"think" + 0.006*"just" + 0.006*"know" + 0.005*"like" + 0.005*"read" + 0.005*"article" + 0.005*"make" + 0.004*"good"
2019-11-13 15:54:11,451 : INFO : topic #4 (0.167): 0.012*"people" + 0.010*"like" + 0.008*"just" + 0.008*"did" + 0.006*"man" + 0.006*"know

2019-11-13 15:54:18,251 : INFO : topic diff=0.007056, rho=0.033251
2019-11-13 15:54:18,305 : INFO : PROGRESS: pass 1, at document #710000/1804874
2019-11-13 15:54:19,829 : INFO : merging changes from 2000 documents into a model of 1804874 documents
2019-11-13 15:54:19,948 : INFO : topic #0 (0.167): 0.015*"money" + 0.015*"tax" + 0.012*"pay" + 0.009*"government" + 0.009*"people" + 0.007*"canada" + 0.007*"state" + 0.007*"taxes" + 0.007*"oil" + 0.006*"cost"
2019-11-13 15:54:19,953 : INFO : topic #2 (0.167): 0.008*"like" + 0.008*"just" + 0.007*"time" + 0.007*"years" + 0.006*"good" + 0.006*"com" + 0.006*"city" + 0.006*"www" + 0.005*"canadians" + 0.005*"http"
2019-11-13 15:54:19,959 : INFO : topic #1 (0.167): 0.042*"trump" + 0.011*"president" + 0.009*"did" + 0.008*"obama" + 0.008*"news" + 0.008*"like" + 0.007*"just" + 0.007*"party" + 0.007*"media" + 0.006*"liberal"
2019-11-13 15:54:19,967 : INFO : topic #5 (0.167): 0.011*"people" + 0.008*"does" + 0.007*"think" + 0.007*"just" + 0.006*"know" + 

KeyboardInterrupt: 

In [ ]:
# Transform the docs from the word space to the topic space (like "transform" in sklearn)
lda_corpus = lda[corpus]

In [ ]:
# Store the documents' topic vectors in a list so we can take a peak
lda_docs = [doc for doc in lda_corpus]